# Préparation des données de l'INSEE

In [1]:
import csv
import json
import cbor2

In [2]:
IN_FILE = "communes-france-2025.csv"
with open(IN_FILE, newline='') as csvfile:
    insee_data = list(csv.DictReader(csvfile))

In [8]:
with open('example_response_2021.json', 'r') as f:
    data_2021 = json.load(f)
previous = dict()
for commune in data_2021["features"]:
    code = commune["properties"]["insee"]
    name = commune["properties"]["name"]
    contributions = commune["properties"]["contributions"]
    previous[code] = contributions

OUT_FILE = "communes_insee.cbor"
communes = list()   
for row in insee_data:
    code = row["code_insee"]
    communes.append((
        code,
        # nom
        row["nom_standard"],
        # population >= 5000
        int(row["population"]) >= 5000,
        # previous contributions, if any
        previous.get(code, 0),
        ))
with open(OUT_FILE, "wb") as f:
    cbor2.dump(communes, f)

In [4]:
dep_codes = dict()
problems = list()
for row in insee_data:
    dep_code = row["dep_code"]
    dep_name = row["dep_nom"]
    if dep_name != dep_codes.get(dep_code, dep_name):
        problems.append((dep_code, dep_name))
        print(f"problem: {dep_code} {dep_name}")
    dep_codes[dep_code] = dep_name
len(dep_codes)
with open('departements.cbor', "wb") as f:
    cbor2.dump(dep_codes, f)

In [7]:
!ls -ltrh *.cbor

-rw-rw-r-- 1 daan daan 651K Mar 19 15:09 codes_insee.cbor
-rw-rw-r-- 1 daan daan 1.4K Mar 19 16:08 departements.cbor
-rw-rw-r-- 1 daan daan 755K Mar 19 16:37 communes_insee.cbor
